In [2]:
import pandas as pd
import os

In [5]:
df=pd.DataFrame()
path='/home/vedant/projects/ScreamDetection/CNN/trained_models/fcnn'
for file_name in os.listdir(path):
    if file_name.endswith(".csv"):
        tmp=pd.read_csv(f'{path}/{file_name}')
        layer1 = file_name[file_name.find('_layer1')+8:file_name.find('_batch')]
        optim = file_name[file_name.find('_optim')+7:file_name.find('_layer1')]
        batch = float(file_name[file_name.find('_batch')+7:file_name.find('_epochs')])
        epochs = int(file_name[file_name.find('_epochs')+8:file_name.find('_lr')])
        lr = float(file_name[file_name.find('_lr')+4:file_name.find('.csv')].replace('_','.'))

        tmp['file_name']=file_name[:-4]
        tmp['optimiser']=optim
        tmp['batch_size']=batch
        tmp['learning_rate']=lr
        tmp['epochs']=epochs
        tmp['layer1_nodes']=layer1

        tmp.drop('Unnamed: 0',axis=1,inplace=True)

        df=df.append(tmp,ignore_index=True)


In [6]:
grp=df.groupby('file_name').max()
grp[grp['validation_acc']==grp['validation_acc'].max()]

,optimiser,epochs,learning_rate,layer1_nodes,batch_size,training_loss,validation_loss,training_acc,validation_acc
file_name,,,,,,,,,
fcnn_optim-adam_layer1-1024_batch-256_epochs-10000_lr-0_01,adam,10000,0.010,1024,256.0,2.595789,22.342693,1.0,0.862893
fcnn_optim-adam_layer1-512_batch-1024_epochs-5000_lr-0_005,adam,5000,0.005,512,1024.0,1.541993,4.372112,1.0,0.862893


In [83]:
from tensorflow import keras
path='/home/vedant/projects/ScreamDetection/CNN/trained_models/fcnn'
model = keras.models.load_model(f'{path}/fcnn_optim-adam_layer1-1024_batch-256_epochs-10000_lr-0_01')
model.summary()

Model: "sequential_383"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_766 (Dense)            (None, 1024)              66560     
_________________________________________________________________
dense_767 (Dense)            (None, 3)                 3075      
Total params: 69,635
Trainable params: 69,635
Non-trainable params: 0
_________________________________________________________________


In [85]:
model = keras.models.load_model(f'{path}/fcnn_optim-adam_layer1-512_batch-1024_epochs-5000_lr-0_005')
model.summary()

Model: "sequential_317"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_634 (Dense)            (None, 512)               33280     
_________________________________________________________________
dense_635 (Dense)            (None, 3)                 1539      
Total params: 34,819
Trainable params: 34,819
Non-trainable params: 0
_________________________________________________________________


In [70]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical

import pandas as pd
import scipy
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import validation_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

import os
import pandas as pd
import soundfile as sf
import numpy as np
import math
import scipy.io.wavfile, scipy.signal
from scipy.spatial import distance
from sklearn.preprocessing import StandardScaler

from sys import platform
if platform == "linux" or platform == "linux2":
    # linux
    path='/home/vedant/projects/'
elif platform == "darwin":
    # OS X
    path='/Users/vedant/Desktop/Programming/'

cols=['video_id', 'start_time', 'mid_ts', 'label', 'average_zcr',
       'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std',
       'centroid_mean','centroid_std',
       'contrast_mean','contrast_std',
       'flatness_mean','flatness_std',
       'rolloff_mean','rolloff_std','rms_mean','rms_std','vggish']
       
d=np.load(path+'ScreamDetection/resources/working_data/vocal_only_features.npy',allow_pickle=True)
df = pd.DataFrame(d,columns=cols)

lut = pd.read_csv(path+'ScreamDetection/resources/dataset/lookup_new.csv')

df.drop('vggish',axis=1,inplace=True)
feature_df=df
mapping=[]
for index,row in feature_df.iterrows():
    if row['label'] == 'clean':
        mapping.append(0)
    if row['label'] == 'highfry':
        mapping.append(1)
    if row['label'] == 'layered':
        mapping.append(1)
    if row['label'] == 'lowfry':
        mapping.append(1)
    if row['label'] == 'midfry':
        mapping.append(1)
    if row['label'] == 'no_vocals':
        mapping.append(2)

feature_df.insert(4,'label_mapped',mapping)

cols=['video_id', 'start_time', 'mid_ts', 'label', 'label_mapped',
       'average_zcr', 'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std', 'centroid_mean', 'centroid_std',
       'contrast_mean', 'contrast_std', 'flatness_mean', 'flatness_std',
       'rolloff_mean', 'rolloff_std', 'rms_mean', 'rms_std']
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='not minority',random_state=0)
from collections import Counter
X = feature_df.to_numpy()
y=feature_df[['label_mapped']].to_numpy()

X_under, y_under = undersample.fit_resample(X, y)

undersampled_data = pd.DataFrame(X_under,columns=cols)
undersampled_data['label_mapped'] = y_under
#print(undersampled_data)


from sklearn.model_selection import GroupShuffleSplit
train_inds, test_inds = next(GroupShuffleSplit(test_size=.2, n_splits=2, random_state = 42).split(lut, groups=lut['band_name']))

train = lut.iloc[train_inds]
test = lut.iloc[test_inds]

train_ids = train['video_id'].to_numpy()
test_ids = test['video_id'].to_numpy()

#df_final = df
df_final = undersampled_data
train = df_final[df_final.video_id.isin(train_ids)]
test = df_final[df_final.video_id.isin(test_ids)]

features=['average_zcr', 'zcr_stddev', 'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std', 'centroid_mean', 'centroid_std',
       'contrast_mean', 'contrast_std', 'flatness_mean', 'flatness_std',
       'rolloff_mean', 'rolloff_std', 'rms_mean', 'rms_std']
x_train = train[features].to_numpy()
y_train_hot = to_categorical(train['label_mapped'].to_numpy())

x_test1 = test[features].to_numpy()
y_test_hot1 = to_categorical(test['label_mapped'].to_numpy())

x_test,X_valid,y_test_hot,y_valid=train_test_split(x_test1,y_test_hot1,test_size=0.2, random_state=42)

X_train=x_train
X_test=x_test

X_train=np.array(X_train)
X_test=np.array(X_test)

scaler = StandardScaler()

X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)


In [86]:
y_pred=model.predict(X_valid)
print(pd.DataFrame(y_pred)[0].unique())
print(pd.DataFrame(y_pred)[1].unique())
print(pd.DataFrame(y_pred)[2].unique())

[0.00000000e+00 2.56778136e-31 9.87762865e-21 6.55810978e-31
 4.12483507e-31 8.76346119e-27 1.48686665e-38 9.40194325e-24
 4.39472270e-14 1.24631192e-29 7.16870881e-32 1.82451418e-24
 4.91523607e-35 1.70443669e-28 1.31438503e-30 4.99524114e-31
 1.31761340e-06 2.47878992e-33 3.41133689e-30 2.85898649e-25
 2.70321383e-31 2.10011782e-29 1.58026823e-25 9.83631710e-26
 3.55435740e-28 1.17609927e-33 2.46374978e-36 6.60391532e-34
 9.09326265e-17 2.98396852e-30 8.77672908e-30 1.75700355e-23
 7.75396849e-33 2.12803741e-36 1.09768927e-17 1.26988802e-32
 1.54071561e-38 1.24797722e-36 1.19339854e-27 1.85725227e-31
 1.46670141e-20 3.02887170e-27 1.29609774e-29 1.72010410e-23
 4.96745472e-33 6.61371114e-30 1.57851285e-35 1.21096324e-33
 2.47258957e-37 2.22561042e-27 9.63570939e-38 1.64696493e-33
 3.50761900e-20 3.20977799e-24 2.12008614e-16 2.36212363e-13
 1.28788541e-16 3.75481695e-31 1.31867421e-37 1.02848864e-32
 7.86988130e-35 1.99824316e-30 8.68859704e-35 4.07524610e-28
 1.70372542e-31 2.785927